# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [45]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [46]:
input_keyword = "網路爬蟲"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果
# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

網路爬蟲: %E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2
/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [66]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


網路爬蟲（英語：web crawler），也叫網路蜘蛛（spider），是一種用來自動瀏覽全球資訊網的網路機器人。其目的一般為編纂網路索引（英語：Web indexing）。

網路搜尋引擎等站點通過爬蟲軟體更新自身的網站內容（英語：Web content）或其對其他網站的索引。網路爬蟲可以將自己所存取的頁面儲存下來，以便搜尋引擎事後生成索引（英語：Index (search engine)）供用戶搜尋。

爬蟲存取網站的過程會消耗目標系統資源。不少網路系統並不默許爬蟲工作。因此在存取大量頁面時，爬蟲需要考慮到規劃、負載，還需要講「禮貌」。 不願意被爬蟲存取、被爬蟲主人知曉的公開站點可以使用robots.txt檔案之類的方法避免存取。這個檔案可以要求機器人（英語：Software agent）只對網站的一部分進行索引，或完全不作處理。

網際網路上的頁面極多，即使是最大的爬蟲系統也無法做出完整的索引。因此在公元2000年之前的全球資訊網出現初期，搜尋引擎經常找不到多少相關結果。現在的搜尋引擎在這方面已經進步很多，能夠即刻給出高品質結果。

爬蟲還可以驗證超連結和HTML代碼，用於網路抓取（英語：Web scraping）（參見資料驅動編程（英語：Data-driven programming））。

網路爬蟲也可稱作網路蜘蛛[1]、螞蟻、自動索引程式（automatic indexer）[2] ，或（在FOAF（英語：FOAF (software)）軟體中）稱為網路疾走（web scutter）。[3]

網路爬蟲始於一張被稱作種子的統一資源位址（URL）列表。當網路爬蟲存取這些統一資源定位器時，它們會甄別出頁面上所有的超連結，並將它們寫入一張「待訪列表」，即所謂爬行疆域（英語：crawl frontier）。此疆域上的URL將會被按照一套策略迴圈來存取。如果爬蟲在執行的過程中複製歸檔和儲存網站上的資訊，這些檔案通常儲存，使他們可以較容易的被檢視。閱讀和瀏覽他們儲存的網站上並即時更新的資訊，這些被儲存的網頁又被稱為「快照」。越大容量的網頁意味著網路爬蟲只能在給予的時間內下載越少部分的網頁，所以要優先考慮其下載。高變化率意味著網頁可能已經被更新或者被取代。一些伺服器端軟體生成的URL（統一資源定位符）也使得網路爬蟲很難避免檢索到重複內容。

但是網際網路的資源卷帙浩繁

### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [100]:
def getExtraUrl(content):
    dictTmp = ({})
    for ext_link in content:
        a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
        if len(a_tag) > 0:
            for link_string in a_tag:
                a_link = link_string["href"]       # 外部連結的網址
                a_keyword = link_string.get_text()  # 外部連結的中文名稱
                tmp={};
                tmp={a_keyword:a_link}
                dictTmp.update(tmp);
    return dictTmp;

In [101]:
getExtraUrl(content)

{'全球資訊網': '/wiki/%E4%B8%87%E7%BB%B4%E7%BD%91',
 '網路機器人': '/wiki/%E7%BD%91%E7%BB%9C%E6%9C%BA%E5%99%A8%E4%BA%BA',
 '網路': '/wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E',
 '搜尋引擎': '/wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E',
 'robots.txt': '/wiki/Robots.txt',
 '網站': '/wiki/%E7%BD%91%E7%AB%99',
 '超連結': '/wiki/%E8%B6%85%E9%80%A3%E7%B5%90',
 'HTML': '/wiki/HTML',
 '網頁': '/wiki/%E7%B6%B2%E9%A0%81',
 '網際網路': '/wiki/%E4%BA%92%E8%81%94%E7%BD%91',
 '伺服器': '/wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8',
 '超文字傳輸協定': '/wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A'}

## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [111]:
def WikiArticle(key_word_link, key_word, recursive):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        tmp='';
        for line in content:
            tmp+=line.get_text()+'\n';
        f = open('./WikiArticle/'+key_word+".txt", "x")
        f.write(tmp)
        f.close()
        print(key_word)

        
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = getExtraUrl(content)


                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                print(k,v)
                WikiArticle(v, k, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [110]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2 (網路爬蟲)
網路爬蟲
全球資訊網 /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
遞迴層[1] - /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91 (全球資訊網)
全球資訊網
網際網路 /wiki/%E5%9B%A0%E7%89%B9%E7%BD%91
遞迴層[2] - /wiki/%E5%9B%A0%E7%89%B9%E7%BD%91 (網際網路)
網際網路
20世紀 /wiki/20%E4%B8%96%E7%B4%80
電腦網路 /wiki/%E7%94%B5%E8%84%91%E7%BD%91%E7%BB%9C
網路 /wiki/%E7%B6%B2%E8%B7%AF
網路協定 /wiki/%E7%BD%91%E7%BB%9C%E5%8D%8F%E8%AE%AE
學術界 /wiki/%E5%AD%B8%E8%A1%93%E7%95%8C
電子 /wiki/%E7%94%B5%E5%AD%90%E6%8A%80%E6%9C%AF
無線 /wiki/%E7%84%A1%E7%B7%9A
光纖網路技術 /wiki/%E5%85%89%E7%BA%96%E7%B6%B2%E8%B7%AF
超文字 /wiki/%E8%B6%85%E6%96%87%E6%9C%AC
全球資訊網 /wiki/%E5%85%A8%E7%90%83%E8%B3%87%E8%A8%8A%E7%B6%B2
電子郵件 /wiki/%E9%9B%BB%E5%AD%90%E9%83%B5%E4%BB%B6
通話 /wiki/%E7%B6%B2%E9%9A%9B%E5%8D%94%E8%AD%B0%E9%80%9A%E8%A9%B1%E6%8A%80%E8%A1%93
檔案分享 /wiki/%E6%AA%94%E6%A1%88%E5%88%86%E4%BA%AB
美國聯邦政府 /wiki/%E7%BE%8E%E5%9C%8B%E8%81%AF%E9%82%A6%E6%94%BF%E5%BA%9C
ARPANET /wiki/ARPANET
TCP/IP /wiki/TCP/IP
分散網路 /wiki/%E5%88%86%E6%95%A3%E7%BD%91%E7%B

CERN
法語 /wiki/%E6%B3%95%E8%AF%AD
英文 /wiki/%E8%8B%B1%E6%96%87
粒子物理學 /wiki/%E7%B2%92%E5%AD%90%E7%89%A9%E7%90%86%E5%AD%B8
全球資訊網 /wiki/%E5%85%A8%E7%90%83%E8%B3%87%E8%A8%8A%E7%B6%B2
瑞士 /wiki/%E7%91%9E%E5%A3%AB
日內瓦 /wiki/%E6%97%A5%E5%85%A7%E7%93%A6
治外法權 /wiki/%E6%B2%BB%E5%A4%96%E6%B3%95%E6%AC%8A
以色列 /wiki/%E4%BB%A5%E8%89%B2%E5%88%97
歐洲 /wiki/%E6%AD%90%E6%B4%B2
物理學 /wiki/%E7%89%A9%E7%90%86%E5%AD%B8
粒子加速器 /wiki/%E7%B2%92%E5%AD%90%E5%8A%A0%E9%80%9F%E5%99%A8
科瓦爾斯基 /wiki/%E7%A7%91%E7%93%A6%E7%88%BE%E6%96%AF%E5%9F%BA
沃納·海森堡 /wiki/%E6%B2%83%E7%B4%8D%C2%B7%E6%B5%B7%E6%A3%AE%E5%A0%A1
詢問計劃 /wiki/ENQUIRE
提姆·柏內茲-李 /wiki/%E6%8F%90%E5%A7%86%C2%B7%E6%9F%8F%E5%85%A7%E8%8C%B2-%E6%9D%8E
羅伯特·卡里奧 /wiki/%E7%BD%97%E4%BC%AF%E7%89%B9%C2%B7%E5%8D%A1%E9%87%8C%E5%A5%A5
超文字 /wiki/%E8%B6%85%E6%96%87%E6%9C%AC
網格計算 /wiki/%E7%B6%B2%E6%A0%BC%E8%A8%88%E7%AE%97
大型強子對撞機 /wiki/%E5%A4%A7%E5%9E%8B%E5%BC%B7%E5%AD%90%E5%B0%8D%E6%92%9E%E6%A9%9F
大型電子正子對撞機 /wiki/%E5%A4%A7%E5%9E%8B%E9%9B%BB%E5%AD%90%E6%AD%A3%E5%AD%90%E5%B0%8D%E6%92%9E%

圖片
照相機 /wiki/%E7%85%A7%E7%9B%B8%E6%9C%BA
鏡子 /wiki/%E9%95%9C%E5%AD%90
望遠鏡 /wiki/%E6%9C%9B%E8%BF%9C%E9%95%9C
顯微鏡 /wiki/%E6%98%BE%E5%BE%AE%E9%95%9C
數位影像 /wiki/%E6%95%B0%E5%AD%97%E5%9B%BE%E5%83%8F
圖像分析 /wiki/%E5%9B%BE%E5%83%8F%E5%88%86%E6%9E%90
圖像處理 /wiki/%E5%9B%BE%E5%83%8F%E5%A4%84%E7%90%86
圖片 /wiki/%E5%9C%96%E7%89%87
相片 /wiki/%E7%85%A7%E7%89%87
影片 /wiki/%E5%BD%B1%E7%89%87
影片 /wiki/%E5%BD%B1%E7%89%87
遞迴層[2] - /wiki/%E5%BD%B1%E7%89%87 (影片)
影片
聲音 /wiki/%E8%81%B2%E9%9F%B3
遞迴層[2] - /wiki/%E8%81%B2%E9%9F%B3 (聲音)
聲音
聲波 /wiki/%E5%A3%B0%E6%B3%A2
介質 /wiki/%E4%BB%8B%E8%B4%A8
氣體 /wiki/%E7%A9%BA%E6%B0%A3
固體 /wiki/%E5%9B%BA%E4%BD%93
液體 /wiki/%E6%B6%B2%E4%BD%93
聽覺 /wiki/%E5%90%AC%E8%A7%89
器官 /wiki/%E5%99%A8%E5%AE%98
波動 /wiki/%E6%B3%A2%E5%8B%95
赫茲 /wiki/%E8%B5%AB%E8%8C%B2
分貝 /wiki/%E5%88%86%E8%B2%9D
聲學 /wiki/%E5%A3%B0%E5%AD%A6
空氣 /wiki/%E7%A9%BA%E6%B0%A3
音響工程 /wiki/%E9%9F%B3%E9%9F%BF%E5%B7%A5%E7%A8%8B
聲音信號處理 /wiki/%E8%81%B2%E9%9F%B3%E4%BF%A1%E8%99%9F%E8%99%95%E7%90%86
建築聲學 /wiki/%E5%BB%BA%E7%AD%91%E5%A3

聖誕節
基督教 /wiki/%E5%9F%BA%E7%9D%A3%E6%95%99
耶穌 /wiki/%E8%80%B6%E7%A9%8C
降生 /wiki/%E8%80%B6%E7%A8%A3%E9%99%8D%E7%94%9F
節日 /wiki/%E7%BE%85%E9%A6%AC%E5%A4%A9%E4%B8%BB%E6%95%99%E8%81%96%E4%BA%BA%E6%9B%86
教會 /wiki/%E6%95%99%E6%9C%83
12月25日 /wiki/12%E6%9C%8825%E6%97%A5
禮儀年曆 /wiki/%E6%95%99%E6%9C%83%E5%B9%B4%E6%9B%86
教派 /wiki/%E5%9F%BA%E7%9D%A3%E6%95%99%E6%B4%BE%E7%B3%BB
將臨期 /wiki/%E5%B0%87%E8%87%A8%E6%9C%9F
聖誕夜 /wiki/%E8%81%96%E8%AA%95%E5%A4%9C
國家 /wiki/%E5%9C%8B%E5%AE%B6
西方國家 /wiki/%E8%A5%BF%E6%96%B9%E4%B8%96%E7%95%8C
基督教文化 /wiki/%E5%9F%BA%E7%9D%A3%E6%95%99%E6%96%87%E5%8C%96
公共假日 /wiki/%E5%85%AC%E5%85%B1%E5%81%87%E6%97%A5
民俗 /wiki/%E6%B0%91%E4%BF%97
公曆新年 /wiki/%E5%85%83%E6%97%A6
聖經 /wiki/%E8%81%96%E7%B6%93
學術 /wiki/%E5%AD%A6%E6%9C%AF
聖母領報 /wiki/%E5%9C%A3%E6%AF%8D%E9%A2%86%E6%8A%A5
發展初期 /wiki/%E5%9F%BA%E7%9D%A3%E6%95%99%E5%8E%86%E5%8F%B2
農神節 /wiki/%E5%86%9C%E7%A5%9E%E8%8A%82
古羅馬 /wiki/%E5%8F%A4%E7%BD%97%E9%A9%AC
冬至 /wiki/%E5%86%AC%E8%87%B3
日照 /wiki/%E6%97%A5%E7%85%A7
西方教會 /wiki/%E8%A5%BF%E6%96

WorldWideWeb
全球資訊網 /wiki/%E5%85%A8%E7%90%83%E8%B3%87%E8%A8%8A%E7%B6%B2
網頁瀏覽器 /wiki/%E7%B6%B2%E9%A0%81%E7%80%8F%E8%A6%BD%E5%99%A8
所見即所得 /wiki/%E6%89%80%E8%A6%8B%E5%8D%B3%E6%89%80%E5%BE%97
網頁編輯器 /wiki/HTML%E7%B7%A8%E8%BC%AF%E5%99%A8
提姆·柏內茲-李 /wiki/%E6%8F%90%E5%A7%86%C2%B7%E6%9F%8F%E5%85%A7%E8%8C%B2-%E6%9D%8E
網頁伺服器 /wiki/%E7%B6%B2%E9%A0%81%E4%BC%BA%E6%9C%8D%E5%99%A8
遞迴層[2] - /wiki/%E7%B6%B2%E9%A0%81%E4%BC%BA%E6%9C%8D%E5%99%A8 (網頁伺服器)
網頁伺服器
超文字傳輸協定 /wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E4%BC%A0%E8%BE%93%E5%8D%8F%E8%AE%AE
HTTP /wiki/HTTP
HTML /wiki/HTML
URL /wiki/URL
Apache /wiki/Apache
IIS /wiki/IIS
Nginx /wiki/Nginx
新聞群組 /wiki/%E6%96%B0%E9%97%BB%E7%BB%84
遞迴層[2] - /wiki/%E6%96%B0%E9%97%BB%E7%BB%84 (新聞群組)
新聞群組
Usenet /wiki/Usenet
全球資訊網 /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
論壇 /wiki/%E8%AE%BA%E5%9D%9B
NNTP /wiki/NNTP
Forté Agent /wiki/Fort%C3%A9_Agent
Opera /wiki/Opera
Outlook Express /wiki/Outlook_Express
Windows Mail /wiki/Windows_Mail
Netscape /wiki/Netscape
Mozilla /wiki/Mozilla
Mozilla Thunderb

Unix
作業系統 /wiki/%E6%93%8D%E4%BD%9C%E7%B3%BB%E7%BB%9F
AT&T /wiki/AT%26T
貝爾實驗室 /wiki/%E8%B4%9D%E5%B0%94%E5%AE%9E%E9%AA%8C%E5%AE%A4
AT＆T /wiki/AT%26T
Multics /wiki/Multics
肯·湯普遜 /wiki/%E8%82%AF%C2%B7%E6%B1%A4%E6%99%AE%E9%80%8A
丹尼斯·里奇 /wiki/%E4%B8%B9%E5%B0%BC%E6%96%AF%C2%B7%E9%87%8C%E5%A5%87
加州大學柏克萊分校 /wiki/%E5%8A%A0%E5%B7%9E%E5%A4%A7%E5%AD%B8%E6%9F%8F%E5%85%8B%E8%90%8A%E5%88%86%E6%A0%A1
柏克萊軟體套件 /wiki/%E6%9F%8F%E5%85%8B%E8%90%8A%E8%BB%9F%E4%BB%B6%E5%A5%97%E4%BB%B6
Novell /wiki/Novell
Linux /wiki/Linux
Minix /wiki/Minix
System V /wiki/System_V
FreeBSD /wiki/FreeBSD
OpenBSD /wiki/OpenBSD
NetBSD /wiki/NetBSD
聖克魯茲作業 /wiki/%E8%81%96%E5%85%8B%E9%AD%AF%E8%8C%B2%E4%BD%9C%E6%A5%AD
AIX /wiki/AIX
HP-UX /wiki/HP-UX
Solaris /wiki/Solaris
IRIX /wiki/IRIX
麻省理工學院 /wiki/%E9%BA%BB%E7%9C%81%E7%90%86%E5%B7%A5%E5%AD%A6%E9%99%A2
奇異 /wiki/%E9%80%9A%E7%94%A8%E7%94%B5%E6%B0%94
道格拉斯·麥克羅伊 /wiki/%E9%81%93%E6%A0%BC%E6%8B%89%E6%96%AF%C2%B7%E9%BA%A5%E5%85%8B%E7%BE%85%E4%BC%8A
布萊恩·柯林漢 /wiki/%E5%B8%83%E8%90%8A%E6%81%A9%C2

Mosaic
網頁瀏覽器 /wiki/%E7%B6%B2%E9%A0%81%E7%80%8F%E8%A6%BD%E5%99%A8
協定 /wiki/%E7%B6%B2%E8%B7%AF%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A
FTP /wiki/FTP
NNTP /wiki/NNTP
Gopher /wiki/Gopher_(%E7%B6%B2%E8%B7%AF%E5%8D%94%E5%AE%9A)
圖形介面 /wiki/%E5%9C%96%E5%BD%A2%E4%BD%BF%E7%94%A8%E8%80%85%E4%BB%8B%E9%9D%A2
WorldWideWeb /wiki/WorldWideWeb
Erwise /wiki/Erwise
ViolaWWW /wiki/ViolaWWW
美國 /wiki/%E7%BE%8E%E5%9C%8B
伊利諾大學厄巴納香檳分校 /wiki/%E4%BC%8A%E5%88%A9%E8%AB%BE%E5%A4%A7%E5%AD%B8%E5%8E%84%E5%B7%B4%E7%B4%8D%E9%A6%99%E6%AA%B3%E5%88%86%E6%A0%A1
NCSA /wiki/%E5%9B%BD%E5%AE%B6%E8%B6%85%E7%BA%A7%E7%94%B5%E8%84%91%E5%BA%94%E7%94%A8%E4%B8%AD%E5%BF%83
馬克·安德森 /wiki/%E9%A9%AC%E5%85%8B%C2%B7%E5%AE%89%E5%BE%B7%E6%A3%AE
埃里克·比納 /wiki/%E5%9F%83%E9%87%8C%E5%85%8B%C2%B7%E6%AF%94%E7%B4%8D
UNIX /wiki/UNIX
X Window /wiki/X_Window
艾爾·高爾 /wiki/%E8%89%BE%E7%88%BE%C2%B7%E9%AB%98%E7%88%BE
國家科學基金會 /wiki/%E5%9B%BD%E5%AE%B6%E7%A7%91%E5%AD%A6%E5%9F%BA%E9%87%91%E4%BC%9A
SGI /wiki/Silicon_Graphics
吉姆·克拉克 /wiki/%E5%90%89%E5%A7%86%C2%B7%E5%85

馬克·安德森
美國 /wiki/%E7%BE%8E%E5%9B%BD
企業家 /wiki/%E4%BC%81%E4%B8%9A%E5%AE%B6%E7%B2%BE%E7%A5%9E
投資者 /wiki/%E6%8A%95%E8%B3%87%E8%80%85
軟體工程師 /wiki/%E8%BD%AF%E4%BB%B6%E5%B7%A5%E7%A8%8B%E5%B8%88
Mosaic /wiki/Mosaic
瀏覽器 /wiki/%E6%B5%8F%E8%A7%88%E5%99%A8
網景通訊公司 /wiki/%E7%BD%91%E6%99%AF%E9%80%9A%E8%AE%AF%E5%85%AC%E5%8F%B8
矽谷 /wiki/%E7%A1%85%E8%B0%B7
風險投資 /wiki/%E9%A3%8E%E9%99%A9%E6%8A%95%E8%B5%84
Andreessen Horowitz /wiki/Andreessen_Horowitz
惠普 /wiki/%E6%83%A0%E6%99%AE
Facebook /wiki/%E8%84%B8%E8%B0%B1
eBay /wiki/EBay
愛荷華州 /wiki/%E7%88%B1%E8%8D%B7%E5%8D%8E%E5%B7%9E
威斯康辛州 /wiki/%E5%A8%81%E6%96%AF%E5%BA%B7%E8%BE%9B%E5%B7%9E
種子 /wiki/%E7%A8%AE%E5%AD%90
伊利諾大學香檳分校 /wiki/%E4%BC%8A%E5%88%A9%E8%AF%BA%E4%BC%8A%E5%A4%A7%E5%AD%A6%E9%A6%99%E6%A7%9F%E5%88%86%E6%A0%A1
電腦科學 /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA%E7%A7%91%E5%AD%A6
學士學位 /wiki/%E5%AD%B8%E5%A3%AB%E5%AD%B8%E4%BD%8D
德克薩斯州 /wiki/%E5%BE%B7%E5%85%8B%E8%90%A8%E6%96%AF%E5%B7%9E
奧斯汀 /wiki/%E5%A5%A7%E6%96%AF%E6%B1%80
IBM /wiki/IBM
國家超級電腦應用中心 /wiki/%E5%9B%BD%E5%A

標記式語言
超文件標示語言 /wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E7%BD%AE%E6%A0%87%E8%AF%AD%E8%A8%80
可延伸標示語言 /wiki/%E5%8F%AF%E6%89%A9%E5%B1%95%E7%BD%AE%E6%A0%87%E8%AF%AD%E8%A8%80
網頁 /wiki/%E7%BD%91%E9%A1%B5
網路應用程式 /wiki/%E7%BD%91%E7%BB%9C%E5%BA%94%E7%94%A8%E7%A8%8B%E5%BA%8F
文字 /wiki/%E6%96%87%E6%9C%AC
桌面出版 /wiki/%E6%A1%8C%E9%9D%A2%E5%87%BA%E7%89%88
堆疊 /wiki/%E5%A0%86%E6%A0%88
troff /wiki/Troff
TeX /wiki/TeX
PostScript /wiki/PostScript
手稿 /wiki/%E6%89%8B%E7%A8%BF
列印 /wiki/%E5%88%97%E5%8D%B0
字型 /wiki/%E5%AD%97%E4%BD%93
排版 /wiki/%E6%8E%92%E7%89%88
高德納 /wiki/%E9%AB%98%E5%BE%B7%E7%BA%B3
數學 /wiki/%E6%95%B0%E5%AD%A6
物理學 /wiki/%E7%89%A9%E7%90%86%E5%AD%A6
電腦科學 /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA%E7%A7%91%E5%AD%A6
LaTeX /wiki/LaTeX
SGML /wiki/SGML
網頁瀏覽器 /wiki/%E7%BD%91%E9%A1%B5%E6%B5%8F%E8%A7%88%E5%99%A8
RDF /wiki/RDF
XForms /wiki/XForms
DocBook /wiki/DocBook
SOAP /wiki/SOAP
語義網 /wiki/%E8%AA%9E%E7%BE%A9%E7%B6%B2
遞迴層[2] - /wiki/%E8%AA%9E%E7%BE%A9%E7%B6%B2 (語義網)
語義網
英語 /wiki/%E8%8B%B1%E8%AF%AD
全球資訊網協會 /wiki/%E4%B8%8

FileNotFoundError: [Errno 2] No such file or directory: './WikiArticle/TCP/IP.txt'